Fetching Webpages with the Requests Library To Demonstrate
===========================================
SqlAlchemy helps you use a database to store and retrieve information from python.  It abstracts the specific storage engine from te way you use it - so it doesn't care if you end up using MySQL, SQLite, or whatever else. In addition, you can use core and the object-relational mapper (ORM) to avoid writing any SQL at all.  The [SQLAlchemy homepage](http://www.sqlalchemy.org/) has lots of good examples and full documentation.

In [6]:
from sqlalchemy import *
import mediacloud, datetime

mc = mediacloud.api.MediaCloud('85de9a93795d3b796b39d902ad16f2838c617b7e61bf9c3425de04f31b3fa2cb')


## Basic SQL Generation
The core library generates SQL for you.  Read more about it on their [expression language tutorial page](http://docs.sqlalchemy.org/en/rel_1_0/core/index.html). Below are some basic examples.

### Creating a Table
Read more about [defining and creating tables](http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html#define-and-create-tables).

In [7]:
# add `echo=True` to see log statements of all the SQL that is generated
echo=True
engine = create_engine('sqlite:///:memory:') # just save the db in memory for now (ie. not on disk)
metadata = MetaData()

queries = Table('queries', metadata,
    Column('id', Integer, primary_key=True),
    Column('keywords', String(400), nullable=False),
    Column('timestamp', DateTime, default=datetime.datetime.now),
    Column('number', Integer),  
)

metadata.create_all(engine) # and create the tables in the database

### Inserting Data
Read more about generating [SQL insert statements](http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html#insert-expressions).

In [8]:
insert_stmt = queries.insert()
str(insert_stmt) # see an example of what this will do

'INSERT INTO queries (id, keywords, timestamp, number) VALUES (:id, :keywords, :timestamp, :number)'

In [9]:
insert_stmt = queries.insert().values(keywords="puppies")
str(insert_stmt)

'INSERT INTO queries (keywords, timestamp) VALUES (:keywords, :timestamp)'

In [76]:
search = 'test'

res = mc.sentenceCount(search, solr_filter=[mc.publish_date_query( datetime.date(2014, 10, 10), datetime.date(2015, 10, 10) ), 'media_sets_id:1' ])
print res['count']
count = res['count']

126605


In [77]:
db_conn = engine.connect()
result = db_conn.execute(insert_stmt)
result.inserted_primary_key # print out the primary key it was assigned

[11]

In [78]:
insert_stmt = queries.insert().values(keywords="kittens")
result = db_conn.execute(insert_stmt)
result.inserted_primary_key # print out the primary key it was assigned

[12]

In [79]:
db_conn = engine.connect()
result = db_conn.execute(insert_stmt)
result.inserted_primary_key

[13]

In [80]:
insert_stmt = queries.insert().values(keywords="kittens")
result = db_conn.execute(insert_stmt)
result.inserted_primary_key 

[14]

In [81]:
db_conn = engine.connect()
result = db_conn.execute(insert_stmt)
result.inserted_primary_key

[15]

### Retrieving Data
Read more about using [SQL select statments](http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html#selecting).

In [82]:
from sqlalchemy.sql import select
select_stmt = select([queries])
results = db_conn.execute(select_stmt)
for row in results:
    print row

(1, u'iguana', datetime.datetime(2015, 12, 11, 14, 20, 46, 274234), 0)
(2, u'robot', datetime.datetime(2015, 12, 11, 14, 20, 55, 528932), 0)
(3, u'puppy', datetime.datetime(2015, 12, 11, 14, 20, 55, 529179), 0)
(4, u'frog', datetime.datetime(2015, 12, 11, 14, 20, 55, 529272), 0)
(5, u'test', datetime.datetime(2015, 12, 11, 14, 31, 11, 640021), 300)
(6, u'test', datetime.datetime(2015, 12, 11, 14, 34, 56, 660415), 300)
(7, u'test', datetime.datetime(2015, 12, 11, 14, 36, 3, 243609), 300)
(8, u'kittens', datetime.datetime(2015, 12, 11, 14, 36, 7, 827952), None)
(9, u'kittens', datetime.datetime(2015, 12, 11, 14, 36, 8, 299632), None)
(10, u'kittens', datetime.datetime(2015, 12, 11, 14, 36, 8, 691426), None)
(11, u'kittens', datetime.datetime(2015, 12, 11, 14, 36, 35, 994353), None)
(12, u'kittens', datetime.datetime(2015, 12, 11, 14, 36, 36, 778391), None)
(13, u'kittens', datetime.datetime(2015, 12, 11, 14, 36, 37, 402159), None)
(14, u'kittens', datetime.datetime(2015, 12, 11, 14, 36, 

In [83]:
select_stmt = select([queries]).where(queries.c.id==1)
for row in db_conn.execute(select_stmt):
    print row

(1, u'iguana', datetime.datetime(2015, 12, 11, 14, 20, 46, 274234), 0)


In [84]:
select_stmt = select([queries]).where(queries.c.keywords.like('%t%'))
for row in db_conn.execute(select_stmt):
    print row

(2, u'robot', datetime.datetime(2015, 12, 11, 14, 20, 55, 528932), 0)
(5, u'test', datetime.datetime(2015, 12, 11, 14, 31, 11, 640021), 300)
(6, u'test', datetime.datetime(2015, 12, 11, 14, 34, 56, 660415), 300)
(7, u'test', datetime.datetime(2015, 12, 11, 14, 36, 3, 243609), 300)
(8, u'kittens', datetime.datetime(2015, 12, 11, 14, 36, 7, 827952), None)
(9, u'kittens', datetime.datetime(2015, 12, 11, 14, 36, 8, 299632), None)
(10, u'kittens', datetime.datetime(2015, 12, 11, 14, 36, 8, 691426), None)
(11, u'kittens', datetime.datetime(2015, 12, 11, 14, 36, 35, 994353), None)
(12, u'kittens', datetime.datetime(2015, 12, 11, 14, 36, 36, 778391), None)
(13, u'kittens', datetime.datetime(2015, 12, 11, 14, 36, 37, 402159), None)
(14, u'kittens', datetime.datetime(2015, 12, 11, 14, 36, 37, 938124), None)
(15, u'kittens', datetime.datetime(2015, 12, 11, 14, 36, 39, 681923), None)


## ORM
You can use their ORM library to handle the translation into full-fledged python objects.  This can help you build the Model for you [MVC](https://en.wikipedia.org/wiki/Model–view–controller) solution.

In [85]:
import datetime
from sqlalchemy import *
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
Base = declarative_base()

### Creating a class mapping
Read more about [creating a mapping](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#declare-a-mapping).

In [86]:
class Query(Base):
    __tablename__ = 'queries'
    id = Column(Integer, primary_key=True)
    keywords = Column(String(400))
    timestamp = Column(DateTime,default=datetime.datetime.now)
    number = Column(Integer,default=0)
    def __repr__(self):
        return "<Query(keywords='%s' number='%s')>" % (self.keywords, self.number)
Query.__table__

Table('queries', MetaData(bind=None), Column('id', Integer(), table=<queries>, primary_key=True, nullable=False), Column('keywords', String(length=400), table=<queries>), Column('timestamp', DateTime(), table=<queries>, default=ColumnDefault(<function <lambda> at 0x10ec08c08>)), Column('number', Integer(), table=<queries>, default=ColumnDefault(0)), schema=None)

### Creating a connection and session
Read more about [creating this stuff](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#creating-a-session).

In [87]:
engine = create_engine('sqlite:///:memory:') # just save the db in memory for now (ie. not on disk)
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
my_session = Session()

### Inserting Data
Read more about [inserting data with an ORM](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#adding-new-objects).

In [88]:
query = Query(keywords="iguana")
query.keywords

'iguana'

In [89]:
my_session.add(query)
my_session.commit()
query.id

1

### Retrieving Data
Read more about [retrieving data from the db](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#querying) via an ORM class.

In [90]:
for q in my_session.query(Query).order_by(Query.timestamp):
    print q

<Query(keywords='iguana' number='0')>


In [91]:
query1 = Query(keywords="robot")
query2 = Query(keywords="puppy")
query3 = Query(keywords="frog")
my_session.add_all([query1,query2, query3])
my_session.commit()

In [92]:
for q in my_session.query(Query).order_by(Query.timestamp):
    print q

<Query(keywords='iguana' number='0')>
<Query(keywords='robot' number='0')>
<Query(keywords='puppy' number='0')>
<Query(keywords='frog' number='0')>


In [93]:
for q in my_session.query(Query).filter(Query.keywords.like('r%')):
    print q

<Query(keywords='robot' number='0')>


In [94]:
for q in my_session.query(Query).filter(Query.number.like(1)):
    print q